In my opinion, it would be a relatively easy thing to simply make this a classification problem, asking whether someone has Parkinson's or not based on their drawings. I will try this as well. 
As suggested by the author of the Kernal who I forked from, Kevin Mader, there are a couple of interesting analyses that could be performed on this dataset:

1. Try and order the curves and get realistic (if possibly inaccurate) trajectories for the pen movement
2. Quantify the pressure by looking at the thickness of the skeleton at specific points.
3. Start to quantify the 'jigglyness' of the motion (fourier analysis of the time series?, differential motion?)

Looking at the curves, this is an interesting image processing problem.

TODO: Map trajectory of path, indexed labelled x, y positions for potential first to last pen movement. A* or other. Dilation grow but restricted to segmented path?
Use this to then correlate jitteryness after plotting index vs x, y . Or measure difference in distance from prvious data points.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["figure.dpi"] = 160
plt.rcParams["font.size"] = 14
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
#plt.rcParams['axes.labelcolor'] = 'white'
plt.style.use('ggplot')
sns.set_style("whitegrid", {'axes.grid': False})

In [ ]:
import numpy as np
from skimage.io import imread
from skimage.util import montage as montage2d
import pandas as pd
from pathlib import Path
data_dir = Path('../input/drawings/')

## Organize Datasets
Here we organize the datasets by directory so we can see the breakdown a bit better

In [ ]:
draw_df = pd.DataFrame({'path': list(data_dir.glob('*/*/*/*.png'))})
draw_df['img_id'] = draw_df['path'].map(lambda x: x.stem)
draw_df['disease'] = draw_df['path'].map(lambda x: x.parent.stem)
draw_df['validation'] = draw_df['path'].map(lambda x: x.parent.parent.stem)
draw_df['activity'] = draw_df['path'].map(lambda x: x.parent.parent.parent.stem)
print(draw_df.shape, 'images loaded')
draw_df.sample(3)

# Show Images

In [ ]:
def fixed_imread(in_path, resize=True):
    """read images, invert and scale them"""
    c_img = 1.0-imread(in_path, as_gray=True)
    max_dim = np.max(c_img.shape)
    if not resize:
        return c_img
    if c_img.shape==(256, 256):
        return c_img
    if max_dim>256:
        big_dim = 512
    else:
        big_dim = 256
        
    out_img = np.zeros((big_dim, big_dim), dtype='float32')
    c_offset = (big_dim-c_img.shape[0])//2
    d_offset = c_img.shape[0]+c_offset
    
    e_offset = (big_dim-c_img.shape[1])//2
    f_offset = c_img.shape[1]+e_offset
    out_img[c_offset:d_offset, e_offset:f_offset] = c_img[:(d_offset-c_offset), :(f_offset-e_offset)]
    return out_img

In [ ]:
fig, m_axs = plt.subplots(2, 2, figsize=(20, 20))
for c_ax, (c_lab, c_rows) in zip(m_axs.flatten(), draw_df.groupby(['activity', 'disease'])):
    prev_img = montage2d(np.stack([fixed_imread(x) for x in c_rows['path'].iloc[0:9]], 0))
    c_ax.imshow(prev_img, cmap='gray')
    c_ax.set_title(' '.join(c_lab))
    c_ax.axis('off')

# Filter and Segment
We can filter and segment the images in order to extract the drawings more clearly as just drawing pixels and noise

In [ ]:
from skimage.filters import threshold_yen as thresh_func
from skimage.filters import median
from skimage.morphology import disk, opening, diamond

def read_and_thresh(in_path, resize=True):
    c_img = fixed_imread(in_path, resize=resize)
    c_img = (255*c_img).clip(0, 255).astype('uint8')
    c_img = median(c_img, disk(2))
    c_thresh = thresh_func(c_img)
    return c_img>c_thresh
fig, m_axs = plt.subplots(2, 2, figsize=(10, 10))
for c_ax, (c_lab, c_rows) in zip(m_axs.flatten(), draw_df.groupby(['activity', 'disease'])):
    prev_img = montage2d(np.stack([read_and_thresh(x) for x in c_rows['path'].iloc[0:9]], 0))
    c_ax.imshow(prev_img, cmap='gray')
    c_ax.set_title(' '.join(c_lab))
    c_ax.axis('off')

In [ ]:
%%time
# run all images
draw_df['thresh_img'] = draw_df['path'].map(lambda x: read_and_thresh(x, resize=False))

In [ ]:
fig, m_axs = plt.subplots(3, 3)
for c_ax, (c_lab, c_row) in zip(m_axs.flatten(), draw_df.sample(9).iterrows()):
    c_ax.imshow(c_row['thresh_img'], cmap='gray')
    c_ax.set_title('{activity} {disease}'.format(**c_row))
    c_ax.axis('off')

## Keep only large enough components
Only keep objects larger than 10% of the total activated pixels. First label each separate object in image and sum the areas for each label identified (that isn't 0). Keep the index if the count is more than 10% of the total. Perform negative sort to have the largest objects with label 1. Replace the old label number with the new ordered id.

In [1]:
from skimage.morphology import label
from skimage.morphology import closing
def label_sort(in_img, cutoff=0.1):
    total_cnt = np.sum(in_img>0)
    lab_img = label(in_img)
    new_image = np.zeros_like(lab_img)
    remap_index = []
    for k in np.unique(lab_img[lab_img>0]):
        cnt = np.sum(lab_img==k) # get area of labelled object
        if cnt>total_cnt*cutoff:
            remap_index+=[(k, cnt)]
    sorted_index = sorted(remap_index, key=lambda x: -x[1]) # reverse sort - largest is first
    for new_idx, (old_idx, idx_count) in enumerate(sorted_index, 1): #enumerate starting at id 1
        new_image[lab_img==old_idx] = new_idx
    return new_image

In [ ]:
fig, m_axs = plt.subplots(3, 3)
for c_ax, (c_lab, c_row) in zip(m_axs.flatten(), draw_df.sample(9).iterrows()):
    clean_img = closing(label_sort(c_row['thresh_img'])>0, disk(2))
    c_ax.imshow(clean_img, cmap='gray')
    c_ax.set_title('{activity} {disease}'.format(**c_row))
    c_ax.axis('off')

In [ ]:
%%time
draw_df['clean_img'] = draw_df['thresh_img'].map(lambda x: closing(label_sort(x)>0, disk(2)))

In [ ]:
from skimage.morphology import skeletonize

fig, m_axs = plt.subplots(3, 3)
for c_ax, (c_lab, c_row) in zip(m_axs.flatten(), draw_df.sample(9).iterrows()):
    skel_img = skeletonize(c_row['clean_img'])
    skel_y, skel_x = np.where(skel_img)
    skel_x = skel_x*1.0/skel_img.shape[1]
    skel_y = skel_y*1.0/skel_img.shape[0]
    
    c_ax.plot(skel_x, skel_y, 'b.')
    c_ax.set_title('{activity} {disease}'.format(**c_row))
    c_ax.axis('off')

### Convert to table
We convert all of the detected skeleton points into a table and combine all of the results together

In [ ]:
all_row_list = []
for _, c_row in draw_df.iterrows():
    skel_img = skeletonize(c_row['clean_img'])
    skel_y, skel_x = np.where(skel_img)
    skel_x = skel_x*1.0/skel_img.shape[1]
    skel_y = skel_y*1.0/skel_img.shape[0]
    for x, y in zip(skel_x, skel_y):
        d_row = dict(**{k: v for k,v in c_row.items() if len(np.shape(v))<1})
        d_row['x'] = x
        d_row['y'] = y
        all_row_list += [d_row]

In [ ]:
all_row_df = pd.DataFrame(all_row_list)
all_row_df.sample(3)

## Show all of the drawings on the same axis
By plotting the skeleton pixels as points and rescaling we can overlay all of the images on top of each other for better visualization. The healthy patients are significantly more consistent than the Parkinson's.

In [ ]:
fig, m_axs = plt.subplots(2, 2, figsize=(30, 30), dpi=72)
for c_ax, (c_lab, c_rows) in zip(m_axs.flatten(), all_row_df.groupby(['activity', 'disease'])):
    for c_id, d_rows in c_rows.groupby('img_id'):
        mean_std = np.mean([d_rows['x'].std(), d_rows['y'].std()])
        c_ax.plot((d_rows['x']-d_rows['x'].mean())/mean_std, 
                  (d_rows['y']-d_rows['y'].mean())/mean_std, '.', label=c_id, ms=0.75)
    c_ax.legend()
    c_ax.set_title(' '.join(c_lab))
    c_ax.axis('off')

## Pen Movement Trajectories 



Spiral set only first

In [ ]:
spiral_draw_df = draw_df[draw_df['activity'] == 'spiral']

In [ ]:
from scipy import ndimage

In [ ]:
k = np.array([[1,1,1],[1,1,1],[1,1,1]])

In [ ]:
c_id = 'V07PE03'
spiral_draw_df.loc[spiral_draw_df['img_id'] == c_id]['clean_img'].values[0].shape
#spiral_draw_df.iloc[2]['clean_img'].shape

In [ ]:
fig, m_axs = plt.subplots()
m_axs.imshow(spiral_draw_df.loc[spiral_draw_df['img_id'] == c_id]['clean_img'].values[0])
#m_axs.imshow(spiral_draw_df.iloc[2]['clean_img'])
plt.show()

Create skeleton for distance map to determine edge points and intersections

In [ ]:
%%time
spiral_draw_df['skel_img'] = spiral_draw_df['clean_img'].map(lambda x: skeletonize(x))

Test on one image

In [ ]:
c_id = 'V07PE03'
#a = np.where(spiral_draw_df.iloc[2]['skel_img'] == True, 1, 0)
a = np.where(spiral_draw_df.loc[spiral_draw_df['img_id'] == c_id]['skel_img'].values[0] == True, 1, 0)

Num Nearest neighbours

In [ ]:
a_nn = ndimage.convolve(a, k, mode='constant', cval=0.0)
a_nn = a_nn * a

In [ ]:
fig, m_axs = plt.subplots(1,2)
m_axs[0].imshow(a)
m_axs[1].imshow(a_nn)
plt.show()

In [ ]:
fig, m_axs = plt.subplots()
plt.imshow(a_nn == 2)
plt.title('Edges')
plt.show()

In [ ]:
fig, m_axs = plt.subplots()
plt.imshow(a_nn == 4)
plt.title('Intersections')
plt.show()

In [ ]:
a_seg = np.where(a_nn == 4, 0, 1) * a #mask a
a_max = label_sort(a_seg, 0.05)

In [ ]:
fig, m_axs = plt.subplots()
plt.imshow(a_max)
plt.show()

Create single pen trajectory by removing branches and joining segments. From here we can start at the furthest distance from the center.

In [ ]:
def nearest_neighbours(in_img):
    a = np.where(in_img == True, 1, 0)
    a_nn = ndimage.convolve(a, k, mode='constant', cval=0.0)
    a_nn = a_nn * a
    return a_nn

In [ ]:
#%%time
#spiral_draw_df['skel_dist_img'] = spiral_draw_df['skel_img'].map(lambda x: nearest_neighbours(x))

Process to remove branches and keep clean skeleton. Find starting point; max distance from 0,0 after mean shift

In [ ]:
def remove_branches(in_img):
    a = np.where(in_img == True, 1, 0)
    a_nn = nearest_neighbours(in_img)
    a_no_branches = np.where(a_nn == 4, 0, 1) * a
    a_keep = label_sort(a_no_branches, 0.05)
    return a_keep

In [ ]:
%%time
spiral_draw_df['clean_skel_img'] = spiral_draw_df['skel_img'].map(lambda x: remove_branches(x))

In [ ]:
# Needs fixing
def get_edges(in_img):
    a = np.where(in_img == True, 1, 0)
    a_nn == 2

In [ ]:
spiral_draw_df.head()

Cleaned up x,y values from spiral only and processed branches

In [ ]:
all_spiral_list = []
for _, c_row in spiral_draw_df.iterrows():
    skel_img = c_row['clean_skel_img']
    skel_y, skel_x = np.where(skel_img)
    skel_x = skel_x*1.0/skel_img.shape[1]
    skel_y = skel_y*1.0/skel_img.shape[0]
    for x, y in zip(skel_x, skel_y):
        d_row = dict(**{k: v for k,v in c_row.items() if len(np.shape(v))<1})
        d_row['x'] = x
        d_row['y'] = y
        all_spiral_list += [d_row]

In [ ]:
all_spiral_row_df = pd.DataFrame(all_spiral_list)
all_spiral_row_df.sample(3)

In [ ]:
#all_spiral_df = all_row_df[all_row_df['activity'] == 'spiral']

In [ ]:
# Shift x and y

for (c_lab, c_rows) in all_spiral_row_df.groupby(['disease']):
    print(c_lab)
    for c_id, d_rows in c_rows.groupby('img_id'):
        #print(c_id)
        mean_std = np.mean([d_rows['x'].std(), d_rows['y'].std()])
        x_norm = (d_rows['x']-d_rows['x'].mean())/mean_std
        y_norm = (d_rows['y']-d_rows['y'].mean())/mean_std
        indices = all_spiral_row_df.loc[all_spiral_row_df['img_id']==c_id].index.values
        #print(indices)
        all_spiral_row_df.loc[indices, 'x_norm'] = x_norm
        all_spiral_row_df.loc[indices, 'y_norm'] = y_norm
        

In [ ]:
all_spiral_row_df.sample(3)

In [ ]:
c_id = 'V10PE03'
d_rows = all_spiral_row_df[all_spiral_row_df['img_id'] == c_id]
fig, m_axs = plt.subplots()
m_axs.plot((d_rows['y']-d_rows['y'].mean())/mean_std, 
              (d_rows['x']-d_rows['x'].mean())/mean_std, '.', label=c_id, ms=0.75)
plt.legend()
plt.show()